In [7]:
import fitz  # PyMuPDF
import requests
import base64
import os
import shutil
from time import sleep
import json

# Utility function to extract images from PDF pages
def extract_pages_as_images(pdf_path, output_folder):
    doc = fitz.open(pdf_path)
    for page_number in range(len(doc)):
        page = doc.load_page(page_number)
        image_list = page.get_images(full=True)
        
        for image_index, img in enumerate(image_list):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]
            image_filename = f"{output_folder}/page_{page_number + 1}_image_{image_index + 1}.{image_ext}"
            with open(image_filename, "wb") as image_file:
                image_file.write(image_bytes)

# Utility function to encode an image in base64 format
def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        image_bytes = image_file.read()
    return base64.b64encode(image_bytes).decode("utf-8")

# Main function to process PDF and perform OCR using GPT-4 Vision
def process_pdf_with_gpt_vision(pdf_path, output_folder, api_key, gpt_model="gpt-4-vision", verbose=False):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Extract images from the PDF
    extract_pages_as_images(pdf_path, output_folder)

    # Get the list of images
    image_files = [os.path.join(output_folder, file) for file in os.listdir(output_folder) if file.endswith(("png", "jpg", "jpeg"))]
    
    if verbose:
        print(f"Extracted {len(image_files)} images from the PDF")

    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {api_key}',
    }

    # Define the default prompt
    prompt = "Extract the text and data as structured JSON from this image."

    # Process each image with GPT-4 Vision
    for idx, image_file in enumerate(image_files):
        if verbose:
            print(f"Processing image {idx + 1} of {len(image_files)}: {image_file}")

        # Encode the image in base64
        image_base64 = encode_image_to_base64(image_file)

        data = {
            "model": gpt_model,
            "prompt": prompt,
            "image": image_base64,
        }

        try:
            response = requests.post("https://api.openai.com/v1/images", headers=headers, json=data)
            result = response.json()

            # Save the JSON response
            json_output_path = os.path.join(output_folder, f"image_{idx + 1}_output.json")
            with open(json_output_path, "w") as json_file:
                json.dump(result, json_file, indent=4)

            if verbose:
                print(f"Output saved: {json_output_path}")

            sleep(5)  # To avoid hitting API rate limits

        except Exception as e:
            print(f"Error processing image {idx + 1}: {e}")

    # Optional: Clean up the image files
    shutil.rmtree(output_folder, ignore_errors=True)
    if verbose:
        print("Processing complete. Cleaned up temporary files.")


In [8]:
import os
pdf_path = "C:\\Users\\sselva\\Downloads\\testddoc1.pdf"
output_folder = "uploads"
api_key = os.getenv('OPENAI_API_KEY')

process_pdf_with_gpt_vision(pdf_path, output_folder, api_key, verbose=True)


Extracted 3 images from the PDF
Processing image 1 of 3: uploads\page_1_image_1.png
Output saved: uploads\image_1_output.json
Processing image 2 of 3: uploads\page_1_image_2.png
Output saved: uploads\image_2_output.json
Processing image 3 of 3: uploads\page_2_image_1.png
Output saved: uploads\image_3_output.json
Processing complete. Cleaned up temporary files.


Extracting images from DOCX 'C:\Users\sselva\Downloads\KO Documents\KO Documents\KS100121_Modify eGroup Approver.docx'...
Modify eGroup Approver/
Administrator/Reviewer

Check approval matrix sheet so that we come to know whose approval is required
Check the type of eGroup
Check for the same in approval matrix sheet so as to whose permission is required
Navigate to bst.golder.com  eAdministration  Setup  employee group setup  eGroup Type (eTime or eExpense and Supervisor or Administrator)
Find name of that eGroup 
Click on eGroup name and select it
Information will be shown at right side in that update name of Primary or Alternate 1 or Alternate 2 or Alternate 3 as per requirement 
Click on ‘save’ icon



In [2]:
import fitz  # PyMuPDF
import os
import shutil
import numpy as np
import torch
import json
from time import sleep
from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM
from transformers.dynamic_module_utils import get_imports
from unittest.mock import patch
import docx  # For DOCX file processing

# Set up device and dtype
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

def load_model_and_processor():
    def fixed_get_imports(filename: str | os.PathLike) -> list[str]:
        if not str(filename).endswith("modeling_florence2.py"):
            return get_imports(filename)
        imports = get_imports(filename)
        return imports

    with patch("transformers.dynamic_module_utils.get_imports", fixed_get_imports):
        model = AutoModelForCausalLM.from_pretrained("microsoft/Florence-2-base", torch_dtype=torch_dtype, trust_remote_code=True).to(device)

    processor = AutoProcessor.from_pretrained("microsoft/Florence-2-base", trust_remote_code=True)
    return model, processor

# Utility function to extract images from PDF pages
def extract_pages_as_images(pdf_path, output_folder):
    doc = fitz.open(pdf_path)
    for page_number in range(len(doc)):
        page = doc.load_page(page_number)
        image_list = page.get_images(full=True)
        
        for image_index, img in enumerate(image_list):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]
            image_filename = f"{output_folder}/page_{page_number + 1}_image_{image_index + 1}.{image_ext}"
            with open(image_filename, "wb") as image_file:
                image_file.write(image_bytes)

# Utility function to extract text from DOCX files
def extract_text_from_docx(docx_path):
    doc = docx.Document(docx_path)
    text_content = []
    for paragraph in doc.paragraphs:
        text_content.append(paragraph.text)
    return "\n".join(text_content)

# Main function to process files and perform OCR using Florence 2B
def process_file_with_florence(file_path, output_folder, verbose=False):
    # Determine file type
    file_extension = os.path.splitext(file_path)[-1].lower()

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    ocr_results = []

    if file_extension == ".pdf":
        # Extract images from the PDF
        extract_pages_as_images(file_path, output_folder)

        # Get the list of images
        image_files = [os.path.join(output_folder, file) for file in os.listdir(output_folder) if file.endswith(("png", "jpg", "jpeg"))]

        # Define the task prompt for OCR
        task_prompt = "<OCR>"

        # Process each image with the Florence 2B model
        for idx, image_file in enumerate(image_files):
            if verbose:
                print(f"Processing image {idx + 1} of {len(image_files)}: {image_file}")

            # Load the image
            image = Image.open(image_file)

            # Run the OCR using the Florence model
            result = run_example(image, task_prompt)

            # Append the result to the OCR results list
            ocr_results.append(result)

            # Save the JSON response for each image (optional)
            json_output_path = os.path.join(output_folder, f"image_{idx + 1}_output.json")
            with open(json_output_path, "w") as json_file:
                json.dump(result, json_file, indent=4)

            if verbose:
                print(f"Output saved: {json_output_path}")

            sleep(5)  # To avoid hitting API rate limits

        # Clean up the image files
        shutil.rmtree(output_folder, ignore_errors=True)

    elif file_extension == ".docx":
        # Extract text from the DOCX file
        docx_text = extract_text_from_docx(file_path)
        ocr_results.append({"text": docx_text})

        # Optionally, save the DOCX text to a JSON file
        json_output_path = os.path.join(output_folder, "docx_output.json")
        with open(json_output_path, "w") as json_file:
            json.dump({"text": docx_text}, json_file, indent=4)

    else:
        raise ValueError(f"Unsupported file type: {file_extension}")

    if verbose:
        print("Processing complete. Cleaned up temporary files.")

    # Return the consolidated OCR results
    return ocr_results

# Define the run_example function to utilize the Florence model
def run_example(image, task_prompt, text_input=None):
    if text_input is None:
        prompt = task_prompt
    else:
        prompt = task_prompt + text_input

    model, processor = load_model_and_processor()

    if task_prompt == '<OCR>':
        image = image.convert("RGB")

    image = np.array(image)

    inputs = processor(text=prompt, images=image, return_tensors="pt").to('cpu', torch.float32)

    generated_ids = model.generate(
        input_ids=inputs["input_ids"].cpu(),
        pixel_values=inputs["pixel_values"].cpu(),
        max_new_tokens=1024,
        early_stopping=False,
        do_sample=False,
        num_beams=3,
    )

    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    print("Generated Text:", generated_text)
    parsed_answer = processor.post_process_generation(
        generated_text,
        task=task_prompt,
        image_size=(image.shape[1], image.shape[0])
    )

    return parsed_answer

# Example usage:
# process_file_with_florence("your_file.pdf", "output_folder", verbose=True)
# process_file_with_florence("your_file.docx", "output_folder", verbose=True)


In [4]:
he = process_file_with_florence("C:\\Users\\sselva\\Downloads\\KO Documents\\KO Documents\\KS100120_Create or Delete eGroup.docx","uploads",verbose=True)

Processing complete. Cleaned up temporary files.


In [5]:
print(he)

[{'text': 'Create or Delete eGroup\n\nCreate eGroup\nCheck approval matrix sheet so that we come to know whose approval is required\nIf approval provided, navigate to bst.golder.com\xa0\uf0e0\xa0eAdministration\xa0\uf0e0\xa0Setup\xa0\uf0e0\xa0employee group setup\xa0\uf0e0\xa0eGroup Type(here for eg., eExpense supervisor)\nClick on eExpense supervisor\nClick on ‘add’ icon at top of the screen\nProvide the short name of e-group mentioned in ticket\nFill in other details like name of e-group (long name of e-group), primary, alt1, alt2, alt3 supervisor names (all provided in the ticket).\nClick on save. Save button is upward beside add user.\n\nDelete eGroup\nCheck approval matrix sheet so that we come to know whose approval is required \nCheck the type of eGroup\nCheck for the same in approval matrix sheet so as to whose permission is required\nNavigate to bst.golder.com\xa0\uf0e0\xa0eAdministration\xa0\uf0e0\xa0Setup\xa0\uf0e0\xa0employee group setup\xa0\uf0e0\xa0eGroup Type(eTime or eE